In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

## Neural Network With Numeric NAICS "Embeddings"
Use a dense layer to encode the numeric NAICS

*This script takes about 1 hour on my MacBook Air*

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
from pathlib import Path
import importlib, pickle
from sklearn import model_selection

In [4]:
import pandas as pd
import os
import re

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from tensorflow.keras.layers import Dense, Dropout, Input, Embedding, Concatenate, Reshape
import tensorflow.keras.metrics as km

In [6]:
from sba_gnn.sba_gnn import sg_plot 

## Input Data

In [7]:
business_data = pd.read_parquet(Path(setup.temp_path).joinpath('10_DATA_combined_scaled_all.parquet'))

In [ ]:
# Previously rescaled NAICS data, sorted
X_naics = pd.read_parquet(Path(setup.temp_path).joinpath('13_NAICS_recode.parquet')) 

In [17]:
X_naics_train = X_naics[X_naics['dset'] == 'train'][['NAICS_rescale']]
X_naics_val = X_naics[X_naics['dset'] == 'val'][['NAICS_rescale']]

In [10]:
with open(Path(setup.temp_path).joinpath('10_DATA_features.pkl'), 'rb') as fin:
    imputer_features = pickle.load(fin)

In [11]:
numeric_features = [f for f in imputer_features if f != 'NAICS']
print(numeric_features)

['NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business', 'urban_flag', 'franchise_flag', 'missingindicator_LowDoc', 'missingindicator_new_business', 'missingindicator_urban_flag']


##### Datasets for train, validation

In [12]:
X = business_data[['dset', 'LoanNr_ChkDgt'] + numeric_features].set_index('LoanNr_ChkDgt').sort_index()

In [13]:
X_train = X[X['dset'] == 'train'].drop(columns='dset')
y_train = business_data[business_data['dset'] == 'train'].set_index('LoanNr_ChkDgt').sort_index()['target']
print(f'training X: {X_train.shape}, y:{y_train.shape}')

training X: (425594, 10), y:(425594,)


In [14]:
X_val = X[X['dset'] == 'val'].drop(columns='dset')
y_val = business_data[business_data['dset'] == 'val'].set_index('LoanNr_ChkDgt').sort_index()['target']
print(f'val X: {X_val.shape}, y:{y_val.shape}')

val X: (126041, 10), y:(126041,)


In [15]:
base_thresh = y_train.mean()
print(base_thresh)

0.20307382152943884


## Function to create model

In [19]:
def create_model(n_feat = len(numeric_features), 
                 naics_emd_dim = setup.nn_naics_embed_size,
                 hidden_size = setup.nn_layer_sizes,
                 activation='tanh', lr=setup.nn_learning_rate,
                 opt_func = setup.nn_optimizer, dropout = setup.nn_dropout,
                embed_layer_name = 'naics_reshape'):
    
    n_layers = len(hidden_size)
    
    features_in = Input(shape=(n_feat,))
    
    # Add a layer for encoding the numeric NAICS
    naics_in = Input(shape=(1,))
    embed_layer = Dense(naics_emd_dim,activation=activation,
                  name=embed_layer_name)(naics_in)

    x = Concatenate(axis=-1)([features_in, embed_layer])
    
    for i in range(n_layers):
        x = Dense(hidden_size[i],activation=activation,
                  name=f'layer_{i:02d}')(x)
        x = Dropout(dropout, name=f'dropout_{i:02d}')(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[features_in, naics_in], outputs=output)
    
    # Compile model
    optimizer = opt_func(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, 
                  metrics=[km.AUC(curve='PR'), km.AUC(curve='ROC')])
    return model

## Create, fit model

In [20]:
this_model = create_model()

2024-01-12 16:42:34.889576: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-12 16:42:34.889595: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-12 16:42:34.889599: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-12 16:42:34.889624: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-12 16:42:34.889636: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [21]:
this_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 16)                19712     ['input_2[0][0]']             
                                                                                                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 naics_reshape (Reshape)     (None, 16)                   0         ['embedding[0][0]']           
                                                                                              

In [ ]:
this_history = this_model.fit([X_train, X_naics_train], y_train,
                              validation_data=([X_val, X_naics_val], y_val),
                              batch_size=setup.nn_batch_size,
                              epochs=setup.nn_epochs)

Epoch 1/20


2024-01-12 16:42:35.379145: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13300/13300 [==============================] - ETA: 0s - loss: 0.4620 - auc: 0.3389 - auc_1: 0.7021

2024-01-12 16:45:21.699346: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13300/13300 [==============================] - 195s 15ms/step - loss: 0.4620 - auc: 0.3389 - auc_1: 0.7021 - val_loss: 0.4557 - val_auc: 0.3692 - val_auc_1: 0.7227
Epoch 2/20
13300/13300 [==============================] - 193s 15ms/step - loss: 0.4545 - auc: 0.3604 - auc_1: 0.7181 - val_loss: 0.4533 - val_auc: 0.3742 - val_auc_1: 0.7268
Epoch 3/20
13300/13300 [==============================] - 191s 14ms/step - loss: 0.4519 - auc: 0.3684 - auc_1: 0.7233 - val_loss: 0.4520 - val_auc: 0.3778 - val_auc_1: 0.7295
Epoch 4/20
13300/13300 [==============================] - 2017s 152ms/step - loss: 0.4506 - auc: 0.3725 - auc_1: 0.7261 - val_loss: 0.4532 - val_auc: 0.3793 - val_auc_1: 0.7298
Epoch 5/20
13300/13300 [==============================] - 267s 20ms/step - loss: 0.4496 - auc: 0.3764 - auc_1: 0.7279 - val_loss: 0.4519 - val_auc: 0.3804 - val_auc_1: 0.7309
Epoch 6/20
13300/13300 [==============================] - 200s 15ms/step - loss: 0.4489 - auc: 0.3782 - auc_1: 0.7293 - val_loss: 0.45

In [ ]:
this_history_df = pd.DataFrame(this_history.history)

In [ ]:
this_history_df

In [ ]:
# Rename columns
try:
    this_history_df.columns = ['_'.join(c.split('_')[0:-1])  \
                               if re.search(r'_\d+$', c) else c for c in this_history_df.columns]
except:
    pass
try:
    cur_col = list(this_history_df.columns)
    this_history_df.columns = [cur_col[0]] + \
        [f'{cur_col[i]}_roc'  if (cur_col[i] == cur_col[i-1]) and 'auc'in cur_col[i] \
         else cur_col[i] for i in range(1, len(cur_col))]
except:
    pass

In [ ]:
this_history_df.columns

In [ ]:
this_history_df.to_csv(Path(setup.temp_path).joinpath('14_REPORT_fit_history.csv'))

In [ ]:
this_history_df[['loss', 'val_loss']].plot()

In [ ]:
this_history_df[['auc', 'val_auc']].plot()

In [ ]:
this_history_df[['auc_roc', 'val_auc_roc']].plot()

In [ ]:
this_model.save(Path(setup.temp_path).joinpath('14_DATA_model.keras'),save_format='tf')

## Predictions on all data

In [ ]:
all_predictions = this_model.predict([X.drop(columns='dset'), 
                                      X_naics.drop(columns='dset')])

In [ ]:
all_predictions_df = pd.DataFrame(all_predictions, index=X.index) \
    .set_axis(['predict_prob'], axis=1) \
    .reset_index() \
    .merge(business_data[['target', 'LoanNr_ChkDgt', 'dset', 'dset_naics_holdout', 'NAICS']], 
           on='LoanNr_ChkDgt')

In [ ]:
all_predictions_df[['predict_prob', 'target']].corr(method='spearman')

##### Threshold Tune & Binary Predictions
Using training probability predictions

In [ ]:
all_pred_train = all_predictions_df[all_predictions_df['dset'] == 'train']

In [ ]:
thresh_tune_data = sg_plot.get_f1_frame(all_pred_train['target'], 
                                        all_pred_train['predict_prob'])

In [ ]:
thresh_tune_data.sort_values('f1', ascending=False, inplace=True)
thresh_tune_data.head(3)

In [ ]:
best_thresh = thresh_tune_data['thresh'].iloc[0]
best_thresh

##### Append binary predictions to probability predictions

In [ ]:
all_predictions_df['predict_bin'] = sg_plot.get_binary_predictions(all_predictions_df['predict_prob'], best_thresh)

In [ ]:
all_predictions_df['predict_bin'].value_counts(normalize=True, dropna=False)

In [ ]:
all_predictions_df.to_parquet(Path(setup.temp_path).joinpath('14_DATA_predictions.parquet'))

## Metrics

In [ ]:
metrics_dset_df = all_predictions_df.groupby('dset') \
    .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
    .reset_index()
metrics_test_df = all_predictions_df[all_predictions_df['dset'] == 'test'] \
    .groupby(['dset', 'dset_naics_holdout']) \
    .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
    .reset_index()
metrics_df = pd.concat([metrics_dset_df, metrics_test_df])
metrics_df.to_csv(Path(setup.temp_path).joinpath('14_REPORT_metrics.csv'), index=True)
metrics_df

## Embeddings
Get the NAICS embeddings for all codes

In [ ]:
embed_layer_name = 'naics_reshape'
#embed_layer_name = 'reshape_4'

In [ ]:
unique_naics_x = business_data.drop_duplicates('NAICS') \
    [['LoanNr_ChkDgt'] + numeric_features].set_index('LoanNr_ChkDgt').sort_index()

In [ ]:
unique_naics_x_naics = business_data.drop_duplicates('NAICS')[['LoanNr_ChkDgt', 'NAICS']] \
    .set_index('LoanNr_ChkDgt').sort_index()

In [ ]:
embed_model = Model(inputs=this_model.inputs, outputs=this_model.get_layer(embed_layer_name).output) 

In [ ]:
embed_out = embed_model.predict([unique_naics_x, unique_naics_x_naics])

In [ ]:
embed_out.shape

In [ ]:
embed_df = pd.DataFrame(embed_out, index=unique_naics_x.index) 
embed_df.columns = [f'emb_{i:03d}' for i in range(len(embed_df.columns))]
embed_df.reset_index(inplace=True)

In [ ]:
embed_df = pd.DataFrame(embed_out, index=unique_naics_x.index)  
embed_df.columns = [f'emb_{i:03d}' for i in range(len(embed_df.columns))]
embed_df = embed_df.reset_index() \
    .merge(business_data[['LoanNr_ChkDgt', 'NAICS', 'NAICS_orig']], how='left', on='LoanNr_ChkDgt')

In [ ]:
embed_df.head(3)

In [ ]:
embed_df.to_parquet(Path(setup.temp_path).joinpath('14_DATA_embeddings.parquet'))